In [ ]:
%%capture #libreries for sagemaker
!pip install tensorflow==2.8.0
!pip install scikit-learn==1.0.2
!pip install nltk==3.2.5
!python3 -m pip install --upgrade pip
!python3 -m pip install --upgrade Pillow
!pip install matplotlib
!pip install pandas

In [ ]:
%%capture
!unzip images.zip -d  images_podcast

In [1]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras import preprocessing
from keras_preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
from keras.layers import Embedding, LSTM, Dropout, Dense, Input, Bidirectional, Flatten, Conv2D, MaxPooling2D, concatenate, Conv1D, MaxPooling1D,  BatchNormalization, Activation, GRU
import keras.backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras.layers.pooling import GlobalAveragePooling1D
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras import regularizers, optimizers

2022-03-12 14:52:14.177971: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-12 14:52:14.178005: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[nltk_data] Downloading package stopwords to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('podcast_final.csv')

In [3]:
df = shuffle(df, random_state = 42)

In [4]:
filepaths = df['filepaths'].tolist() #change the paths according to sagemaker directory
new = []
for el in filepaths:
  path = el.split('/')[-1]
  path= 'images_podcast/' + path
  new.append(path)
df['filepaths'] = new

In [5]:
#df['Primary Genre'] = [[el]for el in df['Primary Genre'].tolist()]
df_train = df[:28279]
df_test = df[28279:]
df_train, df_valid = train_test_split(df_train, shuffle = True, train_size=0.8, random_state = 65)
df_train.reset_index(inplace = True, drop = True)
df_valid.reset_index(inplace = True, drop = True)
df_test.reset_index(inplace = True, drop = True)

In [6]:
stopw = stopwords.words('english')
tokenizer = nltk.RegexpTokenizer(r"\w+")

lemmatizer = WordNetLemmatizer()
def clean_text(text):
  text = text.lower().strip()
  text = tokenizer.tokenize(text)
  clean_text = [w for w in text if w.lower() not in stopw]
  clean_text = " ".join([lemmatizer.lemmatize(w) for w in clean_text])
  return clean_text

In [7]:
df_train['Description'] = [clean_text(el) for el in df_train['Description'].tolist()]
df_valid['Description'] = [clean_text(el) for el in df_valid['Description'].tolist()]
df_test['Description'] = [clean_text(el) for el in df_test['Description'].tolist()]

/tmp/ipykernel_48/439466514.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Description'] = [clean_text(el) for el in df_test['Description'].tolist()]


In [8]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = df['Description'].map(len).max()
EMBEDDING_DIM = 300
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(df['Description'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print('Max len:', MAX_SEQUENCE_LENGTH)

Found 96489 unique tokens.
Max len: 3979


In [9]:
model = load_model('full_net_VGG16finetunedbasebatch32.h5') #load model to resume training from checkpoint

2022-03-12 14:52:31.674066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-12 14:52:31.674106: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-12 14:52:31.674130: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2022-03-12 14:52:31.676016: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
VGG16 = keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape = (224, 224, 3)
)

In [11]:
# Freeze four convolution blocks
for layer in VGG16.layers: #[:15]
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(VGG16.layers):
    print(i, layer.name, layer.trainable) 

0 input_3 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False


In [ ]:
#VGG16 + creates RNN
data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal_and_vertical"), layers.RandomRotation(0.2),]
)

lstm_input = Input(shape=(None,)) #text network
x = Embedding(len(word_index), 300, mask_zero=True, input_length=MAX_SEQUENCE_LENGTH, trainable=False)(lstm_input)
x = GRU(128, return_sequences = True)(x)
x = Dropout(0.3)(x)
x = GRU(128, return_sequences = True)(x)
x = Dropout(0.3)(x)
x = GRU(128, return_sequences = False)(x)
x = Dropout(0.1)(x)
lstm_out = Dense(35, activation = 'relu')(x)

inputs_VGG16 = Input(shape=(224,224,3)) #images network
y = data_augmentation(inputs_VGG16)  # Apply random data augmentation
y = tf.keras.applications.vgg16.preprocess_input(y)
y = VGG16(y)
y = keras.layers.Flatten()(y)
y = Dense(512, activation = 'relu')(y)
y = Dropout(0.1)(y)
vgg16_out =  Dense(35, activation = 'relu')(y)

concat_inp = concatenate([vgg16_out, lstm_out]) #concatenate full network
z = Dense(1024, activation='relu')(concat_inp)
z = Dropout(0.4)(z)
output = Dense(35, activation='softmax')(z)

model = Model(name= 'my_fullnet', inputs=[inputs_VGG16, lstm_input], outputs=[output])

model.summary()

Custom generator

Keras does not support nativaly mixed data inputs, so I had to build a custom generator that feeds batches of data (images + text + labels) to the network.

In [10]:
classes = {'After Shows': 0, 'Animation & Manga': 1, 'Arts': 2, 'Books': 3, 'Business': 4, 'Comedy': 5, 'Design': 6, 'Documentary': 7, 'Education': 8, 'Fashion & Beauty': 9, 'Fiction': 10, 'Food': 11, 'Games': 12, 'Health': 13, 'Hobbies': 14, 'Interviews': 15, 'Kids & Family': 16, 'Music': 17, 'Nature': 18, 'News': 19, 'Non-Profit': 20, 'Personal': 21, 'Pets & Animals': 22, 'Places & Travel': 23, 'Politics': 24, 'Religion': 25, 'Science': 26, 'Sexuality': 27, 'Society & Culture': 28, 'Sports': 29, 'Stand-Up': 30, 'TV & Film': 31, 'Technology': 32, 'True Crime': 33, 'Vehicles': 34}

In [11]:
import random
import pandas as pd
import numpy as np
from glob import glob
from PIL import Image
from keras.preprocessing import image as krs_image

# Create the arguments for image preprocessing
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = df['Description'].map(len).max()
EMBEDDING_DIM = 300
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(df['Description'].values)
word_index = tokenizer.word_index
# Create an empty data generator
datagen = ImageDataGenerator()

def custom_generator(subset, batch_size=32):
  if subset == 'training':
    df = df_train
  elif subset == 'testing':
    df = df_test
  else:
    df = df_valid
  i = 0
  while True:
    batch = {'images': [], 'text': [], 'labels': []}
    for b in range(batch_size):
      if i == len(df):
          i = 0
      # Read image from list and convert to array
      image_path = df.loc[i, "filepaths"]
      image = krs_image.load_img(image_path, target_size=(224, 224))
      image = krs_image.img_to_array(image)

      # Read data from csv using the name of current image
      label = df.loc[i, "Primary Genre"]
      text = df.loc[i, 'Description']
      text = tokenizer.texts_to_sequences(text)
      text = [t[0] for t in text if len(t)!=0]
      padding = [0 for _ in range(MAX_SEQUENCE_LENGTH-len(text))]
      text = text + padding
      #text = pad_sequences(text, maxlen=MAX_SEQUENCE_LENGTH)
      
      batch['images'].append(image)
      batch['text'].append(text)
      batch['labels'].append(classes[label])

      i += 1
    batch['images'] = np.array(batch['images'])
    batch['text'] = np.array(batch['text'])
    # Convert labels to categorical values
    batch['labels'] = np.eye(35)[batch['labels']]

    yield [batch['images'], batch['text']], batch['labels']

In [12]:
model.compile(optimizer=keras.optimizers.Adam(), loss="categorical_crossentropy",metrics=["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(5)])

In [13]:
#training VGG16 + custom RNN
CSV_log = tf.keras.callbacks.CSVLogger('full_net_VGG16finetunedbasebatch32.csv', separator=",", append=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=6,
restore_best_weights=True, monitor='val_loss')
model_checkpoint = keras.callbacks.ModelCheckpoint('full_net_VGG16finetunedbasebatch32.h5', monitor='val_loss', save_best_only = True, verbose = 1)

STEP_SIZE_TRAIN=len(df_train)//32
STEP_SIZE_VALID=len(df_valid)//32

history = model.fit(custom_generator('training'), validation_data=(custom_generator('validation')), 
                    epochs=100, callbacks=[early_stopping_cb, model_checkpoint, CSV_log], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps = STEP_SIZE_VALID,
                    initial_epoch = 16
                   )

Epoch 17/100
706/706 [==============================] - ETA: 0s - loss: 2.1129 - accuracy: 0.4192 - top_k_categorical_accuracy: 0.7027
Epoch 17: val_loss improved from inf to 2.40394, saving model to full_net_VGG16baseGRU3.h5
706/706 [==============================] - 875s 1s/step - loss: 2.1129 - accuracy: 0.4192 - top_k_categorical_accuracy: 0.7027 - val_loss: 2.4039 - val_accuracy: 0.3784 - val_top_k_categorical_accuracy: 0.6294
Epoch 18/100
706/706 [==============================] - ETA: 0s - loss: 2.0463 - accuracy: 0.4341 - top_k_categorical_accuracy: 0.7133
Epoch 18: val_loss improved from 2.40394 to 2.36450, saving model to full_net_VGG16baseGRU3.h5
706/706 [==============================] - 858s 1s/step - loss: 2.0463 - accuracy: 0.4341 - top_k_categorical_accuracy: 0.7133 - val_loss: 2.3645 - val_accuracy: 0.3839 - val_top_k_categorical_accuracy: 0.6444
Epoch 19/100
706/706 [==============================] - ETA: 0s - loss: 1.9902 - accuracy: 0.4480 - top_k_categorical_accura

In [ ]:
history_eval = model.evaluate(custom_generator('testing'), steps = len(df_test)/32)

In [25]:
import json
history = {'loss': history_eval[0], 'accuracy': history_eval[1], 'top_5_accuracy': history_eval[2]}
print(history)
json.dump(history, open('fullnet_eval.json', 'w'))

{'loss': 2.3517343997955322, 'accuracy': 0.4140625, 'top_5_accuracy': 0.6787109375}


In [12]:
filenames = df_test['filepaths'].tolist()
real_genre = df_test['Primary Genre'].tolist()
nb_samples = len(filenames)

predict = model.predict(custom_generator('testing'), verbose = 1, steps = nb_samples/32)

31/31 [==============================] - 205s 6s/step


In [31]:
print(len(predict))
print(predict[-1])
print(predict[1])

1024
[1.79916248e-02 1.20362570e-03 1.14894975e-02 6.81950769e-04
 1.73103008e-02 3.36533342e-03 8.65048263e-04 7.91200087e-04
 6.13070419e-03 3.95816658e-03 4.22820728e-03 3.19670234e-03
 3.68652418e-02 2.42632348e-03 7.61923790e-02 1.67034306e-02
 1.63473887e-03 4.84089106e-02 1.63522270e-02 1.88836604e-01
 3.91481997e-04 3.19714006e-03 8.42044130e-04 6.91072317e-03
 5.61812660e-03 5.26376534e-03 7.44604319e-03 9.65712708e-04
 2.71364255e-03 1.45787537e-01 2.40173982e-03 2.24964805e-02
 2.05191955e-01 4.55174813e-05 1.32095873e-01]
[2.62625832e-02 9.09278751e-04 7.44562373e-02 2.79318038e-02
 1.18500402e-03 2.47209799e-02 1.51111846e-04 1.38885016e-02
 5.84705966e-03 1.06852902e-02 5.24273142e-02 3.21218814e-03
 6.30640239e-03 1.45438127e-03 6.87498553e-03 2.29788914e-01
 5.60570695e-02 2.11874261e-01 6.07878622e-03 4.33054268e-02
 2.21823552e-03 2.30649021e-02 1.16882824e-04 1.71868084e-03
 1.20080104e-02 2.56573316e-02 1.07142162e-02 1.37572165e-03
 4.96509671e-02 5.03333798e-03 4.

In [37]:
predicted_class_indices=np.argmax(predict,axis=1)
print(len(predicted_class_indices))
print(predicted_class_indices[:10])
print(predicted_class_indices[1000:1010])

1024
[32 15 10 14 17 10 31  4 34 33]
[32 15 10 14 17 10 31  4 34 33]


In [24]:
print(df_test[:1])

                                      Name  \
0  RN Breakfast - Separate stories podcast   

                                             Artwork Primary Genre  \
0  https://is5-ssl.mzstatic.com/image/thumb/Podca...          News   

                                         Description  \
0  rn breakfast daily story separated easy listen...   

                        filepaths  
0  images_podcast/id124116392.jpg  


In [38]:
labels = dict((v,k) for k,v in classes.items())
predictions = [labels[k] for k in predicted_class_indices]

In [39]:
print(len(filenames))

1000


In [40]:
print(len(predictions))

1024


In [61]:
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions[:1000], 'real genre':real_genre, 'description': df[28279:]['Description'].tolist()})

In [62]:
results.head(9)

,Filename,Predictions,real genre,description
0,images_podcast/id124116392.jpg,Technology,News,RN Breakfast daily stories separated out for e...
1,images_podcast/id1495557557.jpg,Interviews,Music,"For over 4 decades, Bob Clearmountain has defi..."
2,images_podcast/id1552114238.jpg,Fiction,TV & Film,If you’ve never watched Little House on the Pr...
3,images_podcast/id1545221046.jpg,Hobbies,Business,No-bullsh*t strategies to help you *CATAPULT* ...
4,images_podcast/id285097604.jpg,Music,Music,Presented by Mark Knight: 2 hours of the hotte...
5,images_podcast/id1525926766.jpg,Fiction,Fiction,"The Adventures of Sam Spade, Detective was a r..."
6,images_podcast/id1578559817.jpg,TV & Film,TV & Film,It’s one of the wildest scandals in Hollywood ...
7,images_podcast/id1551541889.jpg,Business,Politics,"Washington, D.C.‘s fiercest independent report..."
8,images_podcast/id1051566354.jpg,Vehicles,Politics,Justin Robert Young announces his intention to...
